In [11]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False
%matplotlib inline

In [12]:
df = pd.read_csv("/opt/app/data/workspace/GEPrediction-OSRS/data/rsbuddy/buy_average.csv")
df = df.set_index('timestamp')
df2 = pd.read_csv("/opt/app/data/workspace/GEPrediction-OSRS/data/rsbuddy/sell_average.csv")
df2 = df2.set_index('timestamp')

In [22]:
dropped = df.drop_duplicates()
dropped2 = df2.drop_duplicates()
selected = dropped[['Arcane_spirit_shield', "Inquisitor's_mace",'Sanguinesti_staff_(uncharged)', 'Dragon_claws', 'Sarachnis_cudgel', '3rd_age_full_helmet', '3rd_age_robe', 'Enhanced_crystal_teleport_seed','Scythe_of_vitur_(uncharged)','Old_school_bond']]
selected = selected.replace(to_replace=0, method='ffill')
selected2 = dropped2[['Arcane_spirit_shield', "Inquisitor's_mace",'Sanguinesti_staff_(uncharged)', 'Dragon_claws', 'Sarachnis_cudgel', '3rd_age_full_helmet', '3rd_age_robe', 'Enhanced_crystal_weapon_seed','Scythe_of_vitur_(uncharged)','Old_school_bond']]
# y = runes.reset_index()['Chaos_rune']

KeyError: "['Enhanced_crystal_weapon_seed'] not in index"

In [14]:
print(selected.shape)
print(selected2.shape)
selected.head()

(1, 8)
(1, 8)


,Arcane_spirit_shield,Inquisitor's_mace,Sanguinesti_staff_(uncharged),Dragon_claws,Sarachnis_cudgel,3rd_age_full_helmet,3rd_age_robe,Scythe_of_vitur_(uncharged)
timestamp,,,,,,,,
1628048683,0,0,0,0,0,0,0,0


# Using the tsfresh library

In [17]:
import scipy as sp
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from sklearn.ensemble import AdaBoostRegressor
from tsfresh.utilities.dataframe_functions import impute

In [18]:
selected['Rune_axe'].plot(figsize=(15, 6))
plt.show()

KeyError: 'Rune_axe'

In [19]:
df_shift, y = make_forecasting_frame(selected["Rune_axe"], kind="buy_average", max_timeshift=20, rolling_direction=1)

KeyError: 'Rune_axe'

In [ ]:
df_shift.head()

In [ ]:
df_shift.shape

In [ ]:
%%capture
X = extract_features(df_shift, column_id="id", column_sort="time", column_value="value", impute_function=impute, show_warnings=False)

In [ ]:
# drop constant features
print(X.shape)
X = X.loc[:, X.apply(pd.Series.nunique) != 1] 
print(X.shape)

In [ ]:
# Add last value as feature
X["feature_last_value"] = y.shift(1)

In [ ]:
# Drop first line
X = X.iloc[1:, ]
y = y.iloc[1: ]

In [ ]:
X.head()

In [ ]:

from tqdm import tqdm

In [ ]:
ada = AdaBoostRegressor(n_estimators=10)
y_pred = [np.NaN] * len(y)

isp = 100   # index of where to start the predictions
assert isp > 0

for i in tqdm(range(isp, len(y))):
    
    ada.fit(X.iloc[:i], y[:i])
    y_pred[i] = ada.predict(X.iloc[i, :].values.reshape((1, -1)))[0]
    
y_pred = pd.Series(data=y_pred, index=y.index)

In [ ]:

# Dataframe of predictions and true values
ys = pd.concat([y_pred, y], axis = 1).rename(columns = {0: 'pred', 'value': 'true'})

# Convert index to a datetime
ys.index = pd.to_datetime(ys.index)
ys.head()

In [ ]:
ys.plot(figsize=(15, 8))
plt.title('Predicted and True Price')
plt.show()

In [ ]:
# Create column of previous price
ys['y-1'] = ys['true'].shift(1)
ys[['y-1', 'true']].plot(figsize = (15, 8))
plt.title('Benchmark Prediction and True Price')
plt.show()

In [ ]:
print("MAE y-1: \t{}".format(np.mean(np.abs(np.diff(y))[isp-1:] )))
print("MAE ada: \t{}".format(np.mean(np.abs(y_pred - y)[isp:])))

In [ ]:
importances = pd.Series(index=X.columns, data=ada.feature_importances_)
importances.sort_values(ascending=False).head(10)

# Using known finance features

In [ ]:
def moving_average_convergence(group, nslow=26, nfast=12):
    emaslow = group.ewm(span=nslow, min_periods=1).mean()
    emafast = group.ewm(span=nfast, min_periods=1).mean()
    result = pd.DataFrame({'MACD': emafast-emaslow, 'emaSlw': emaslow, 'emaFst': emafast})
    return result

def moving_average(group, n=9):
    sma = group.rolling(n).mean()
    return sma

In [2]:
print(selected.shape)
print(selected2.shape)
selected.head()

NameError: name 'selected' is not defined

In [3]:
myResult = moving_average_convergence(selected['Rune_axe'])
sma = moving_average(selected['Rune_axe'])

NameError: name 'moving_average_convergence' is not defined

In [4]:
print(myResult.shape)
myResult.head()

NameError: name 'myResult' is not defined

In [5]:
sma.head(15)

NameError: name 'sma' is not defined

In [6]:
def RSI(price, n=14):
    delta = price.diff()
    dUp, dDown = delta.copy(), delta.copy()
    dUp[dUp < 0] = 0
    dDown[dDown > 0] = 0

    RolUp = dUp.rolling(n).mean()
    RolDown = dDown.rolling(n).mean().abs()
    
    RS = RolUp / RolDown
    rsi= 100.0 - (100.0 / (1.0 + RS))
    return rsi

In [7]:
rsi = RSI(selected['Rune_axe'], 10)
rsi = rsi.rename("RSI")
rsi.head(15)

NameError: name 'selected' is not defined

In [8]:
selected['datetime'] = selected.index
selected['datetime'] = pd.to_datetime(selected['datetime'], unit='s')
selected['dayofweek'] = selected['datetime'].dt.dayofweek
selected['hour'] = selected['datetime'].dt.hour
selected['minute'] = selected['datetime'].dt.minute

selected.head()

NameError: name 'selected' is not defined

In [9]:
tmp = selected.copy()
tmp.index = pd.to_datetime(tmp.index)

slope = pd.Series(np.gradient(tmp['Rune_axe']), tmp.index, name='slope')

df = pd.concat([tmp, slope], axis=1)
df.head()

NameError: name 'selected' is not defined

In [10]:
from scipy import integrate

def integrate_method(self, how='trapz', unit='s'):
    available_rules = set(['trapz', 'cumtrapz', 'simps', 'romb'])
    if how in available_rules:
        rule = integrate.__getattribute__(how)
    else:
        print('Unsupported integration rule: %s' % (how))
        print('Expecting one of these sample-based integration rules: %s' % (str(list(available_rules))))
        raise AttributeError
    
    result = rule(self.values, self.index.astype(np.int64) / 10**9)
    #result = rule(self.values)
    return result

pd.Series.integrate = integrate_method

integral = pd.Series(tmp['Rune_axe'].integrate('cumtrapz'))
integral.head()

ModuleNotFoundError: No module named 'scipy'

In [11]:
# Univariate linear regression tests and Recursive Feature Elimination (RFE)
from sklearn import datasets
from sklearn.feature_selection import RFE, f_regression, SelectKBest
from sklearn.linear_model import LogisticRegression

ModuleNotFoundError: No module named 'sklearn'

## Regression F Test

In [12]:
# generate dataset
X, y = datasets.make_regression(n_samples=100, n_features=100, n_informative=10)
print(X.shape)
print(y.shape)

# define feature selection
fs = SelectKBest(score_func=f_regression, k=10)
# apply feature selection
X_selected = fs.fit_transform(X, y)
print(X_selected.shape)

NameError: name 'datasets' is not defined

In [13]:
features = df.drop(['datetime'], axis=1).copy()

# normalize dataset
dataset=(features-features.mean())/features.std()

X = dataset.drop(['Rune_axe'], axis=1)
y = dataset['Rune_axe']
print(X.shape)
print(y.shape)

# define feature selection
fs = SelectKBest(score_func=f_regression, k=3)
# apply feature selection
X_selected = fs.fit_transform(X, y)
print(X_selected.shape)

# Get scores for each of the columns
scores = fs.scores_
print(scores)

# Get columns to keep and create new dataframe with those only
cols = fs.get_support(indices=True)
features_df_new = X.iloc[:,cols]
features_df_new.head()


NameError: name 'df' is not defined

## Recursive Feature Elimination (RFE)

In [14]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from matplotlib import pyplot

ModuleNotFoundError: No module named 'sklearn'

In [15]:
X = dataset.drop(['Rune_axe'], axis=1)
y = dataset['Rune_axe']

# perform feature selection
rfe = RFE(RandomForestRegressor(n_estimators=500, random_state=1), 4)
fit = rfe.fit(X, y)
# report selected features
print('Selected Features:')
names = dataset.drop(['Rune_axe'], axis=1).columns.values
for i in range(len(fit.support_)):
    if fit.support_[i]:
        print(names[i])
        
# plot feature rank
names = dataset.drop(['Rune_axe'], axis=1).columns.values
ticks = [i for i in range(len(names))]
pyplot.bar(ticks, fit.ranking_)
pyplot.xticks(ticks, names)
pyplot.show()

NameError: name 'dataset' is not defined